In [1]:
import pyedflib
import numpy as np
import matplotlib.pyplot as plt

In [2]:
file_name = "chb01_01.edf"
f = pyedflib.EdfReader(file_name)
n = f.signals_in_file
signal_labels = f.getSignalLabels() 
sigbufs = np.zeros((n, f.getNSamples()[0]))
for i in np.arange(n):
        sigbufs[i, :] = f.readSignal(i)
# section each data in sigbufs into 20 seconds
# for signal in sigbufs:
#         for i in range(0, len(signal), 2000):
#                 print(signal[i:i+4000])
#                 plt.plot(signal[i:i+4000])
#                 plt.show()

In [3]:
def read_file(file_name):
    f = pyedflib.EdfReader(file_name)
    n = f.signals_in_file
    signal_labels = f.getSignalLabels() 
    sigbufs = np.zeros((n, f.getNSamples()[0]))
    for i in np.arange(n):
            sigbufs[i, :] = f.readSignal(i)

In [18]:
def partition_signals_in_20_seconds(sigbufs):
    #section each data in sigbufs into 20 seconds data and return the list of 20 seconds data
    signals_in_20_seconds = []
    for signal in sigbufs:
        for i in range(0, len(signal), 20000):
            signals_in_20_seconds.append(signal[i:i+20000])
    return signals_in_20_seconds

In [19]:
new_sigbufs = partition_signals_in_20_seconds(sigbufs)

In [24]:
len(new_sigbufs)

1081

## a convolution neural network with tensorflow class 


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

class CNN:
    def __init__(self, learning_rate=0.001, num_classes=2, batch_size=50, dropout=0.75, num_epochs=10, evaluate_every=100, checkpoint_every=100, allow_soft_placement=True, log_device_placement=False):
        self.learning_rate = learning_rate
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.dropout = dropout
        self.num_epochs = num_epochs
        self.evaluate_every = evaluate_every
        self.checkpoint_every = checkpoint_every
        self.allow_soft_placement = allow_soft_placement
        self.log_device_placement = log_device_placement

    def train(self, train_data, train_labels, test_data, test_labels):
        # create a graph to hold the model
        self.graph = tf.Graph()
        with self.graph.as_default():
            self.session_conf = tf.ConfigProto(allow_soft_placement=self.allow_soft_placement, log_device_placement=self.log_device_placement)
            self.sess = tf.Session(config=self.session_conf)
            with self.sess.as_default():
                # create the model
                self.cnn()
                # define training procedure
                self.global_step = tf.Variable(0, name="global_step", trainable=False)
                self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
                self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
                self.train_op = self.optimizer.apply_gradients(self.grads_and_vars, global_step=self.global_step)
                # keep track of gradient values and sparsity
                self.grad_summaries = []
                for g, v in self.grads_and_vars:
                    if g is not None:
                        self.grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                        self.sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                        self.grad_summaries.append(self.grad_hist_summary)
                        self.grad_summaries.append(self.sparsity_summary)
                self.grad_summaries_merged = tf.summary.merge(self.grad_summaries)
                # output directory for models and summaries
                self.timestamp = str(int(time.time()))
                self.out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", self.timestamp))
                print("Writing to {}\n".format(self.out_dir))
                # summaries for loss and accuracy
                self.loss_summary = tf.summary.scalar("loss", self.loss)
                self.acc_summary = tf.summary.scalar

## a convolution neural network class with pytorch


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x